In [1]:
using DataStructures
using FileIO
using Meshes
using MeshIO
using Makie
using GLMakie
using GeometryBasics
using ProgressBars
using JLD2
using Serialization

In [21]:
include("contractions.jl")
include("visualisation.jl")
include("preprocessing.jl")

loadsc (generic function with 1 method)

In [40]:
bni = deserialize("bunidata.dat")
for vertex in bni.coords
    # rotate by 90 degrees around the x-axis
    vertex[2], vertex[3] = vertex[2] * cos(π / 2) - vertex[3] * sin(π / 2), vertex[2] * sin(π / 2) + vertex[3] * cos(π / 2)
end
include("visualisation.jl")
#visualize(bni)
visualizeNormals(bni)

UndefVarError: UndefVarError: `face_center` not defined in `GeometryBasics`
Suggestion: check for spelling errors or missing imports.

In [ ]:
bunidata = deserialize("bunidata.dat")
buni = initialContractedSimplicialComplex2D(bunidata)
fillholes!(buni.contracted)
calculateFundamentalQuadratics!(buni)

pq = PriorityQueue()
for (edge, triangle) in ProgressBar(buni.contracted._edge_to_triangles)
    pq[edge] = error(buni, edge)
end

In [ ]:
fig = Figure(size=(800, 600))
ax = Axis3(fig[1, 1])

while pq[first(first(pq))] < 0.8
    myedge = dequeue!(pq)
    while !issafe(buni.contracted, myedge)
        myedge = dequeue!(pq)
    end

    new_vertex = contract!(buni, myedge)
    for edge in buni.contracted._vertex_to_edges[new_vertex]
        e = error(buni, edge)
        pq[edge] = e
    end

    if log10(abs(pq[first(first(pq))])) - lastlog > 1
        global lastlog
        global img_count
        d_mesh = scx2mesh(buni.contracted)
        mesh!(ax, d_mesh)
        wireframe!(ax, d_mesh, color=:black)
        ax.azimuth[] = start_angle + 2pi * img_count / antispeed
        recordframe!(io)  # record a new frame
        #ax.scene[:children] = [] # clear Axis3

        img_count += 1
        lastlog = log10(abs(pq[first(first(pq))]))
    end
end